# CS 375 Homework 3: Logistic Regression 

The goal of this assignment is to give you practice implementing a `Logistic Regression` classifier and evaluate the classifier's performance on real-world datasets. 

In this assignment, you can assume the class labels (y) are **binary**. 

*Optional but recommended:* We will use `numpy` for this assignment. If you have not used numpy before or would like a refresher, please review `numpy_tutorial.ipynb` in this repository before beginning the assignment. 

## Organization and Instructions 

Execute the code cells in Part 1 to understand the background for this assignment. You will not need to modify or add anything to Part 1. Part 2 is where your solution begins. 

**Part 1: Background.**
- 1A. Environment set-up 
- 1B. Data exploration 

**Part 2: Your implementation.** This is where you will implement your solution by modifying the following:
- `logistic_function()`
- `NLLLoss()`
- `gradient_nll()` 
- Implementing the `LogisticRegressionClassifier` class and its methods: 
    - `__init__()`
    - `train()`
    - `predict()` 
    - `get_weights()` 

**Part 3: Evaluation on real datasets.**
- 3A. You will train and evaluate on the `triage` data. 
- 3B. You will examine the weights of your trained classifer.
- 3C. You will contemplate tradeoffs between Logistic Regression and Naive Bayes classifiers and write a response.  

**(Optional) Part 4: Extra Credit.** Extra credit can only help you and will not hurt you. At the end of the semester, if you have a borderline grade, your grade could be increased given your efforts on extra credit. This section is intended to be open-ended and challenge you. We suggest you only attempt this section after you have completed all other parts and are satisifed with your submission. 

**Addtional instructions.** 
- Your submitted solution and code must be yours alone. Copying and pasting a solution from the internet or another source is considered a violation of the honor code. 
- However, you can talk to classmates about *high-level* approaches. In the **Process Reporting** section, record the names of any classmates you spoke with about this assignment. 

**Evaluation.**

Your solution will be evaluated on accuracy on the train, dev and test sets, similar to HW2. Our reference implemenation obtained the following accuracies on the `triage` dataset: 
- `dev`: 0.763
- `test`: 0.782

However, we're going to be a bit more generous for this assignment and place `target minimum accuracies` as 
- `dev`: 0.7
- `test`: 0.7 

For logistic regression, we're not grading the training set accuracy because it could vary depending on randomness from stochastic gradient descent. 

**Grading.**

- **10 points (autograded):** `logistic_fucntion()` unit tests. 

- **10 points (autograded):** `NLLLoss()` unit tests.


- **10 points (autograded):** `gradient_nll()` unit tests.
    
- **10 points (autograded):** This portion of your grade reflects how well your submission performs on the `dev set` of the `triage` dataset compared to our reference implementation metrics.  Your points are calculated as 
    ```
    (1 -(0.7 - min(accuracy on dev, 0.7))/0.7) * 10 points 
    ```
- **10 points (autograded):** This portion of your grade reflects how well your submission performs on the `test set` of the `triage` dataset compared to our reference implementation metrics. You will not have access to the test set but will be able to see your score on Gradescope. Your points are calculated as   
    ```
    (1-( 0.7 - min(accuracy on test,  0.7))/ 0.7) * 10 points 
    ```  
    
- **5 points (manually graded):** The TAs and instructor will manually grade your answer to Part 3C. 

**Submission.** 
Once you have completed Parts 1, 2 and 3, run the final cell in this notebook. This will create `submission.zip` which you will then upload to Gradescope. 

## 1A. Environment set-up

If you set-up your conda environment correctly in HW0, you should see `Python [conda env:cs375]` as the kernel in the upper right-hand corner of the Jupyter webpage you are currently on. Run the cell below to make sure your environment is correctly installed. 

In [ ]:
# Environment check 
# Return to HW0 if you run into errors in this cell 
# Do not modify this cell 
import os
assert os.environ['CONDA_DEFAULT_ENV'] == "cs375"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 8

If there are any errors after running the cell above, return to the instructions from `HW0`. If you are still having difficulty, reach out to the instructor or TAs via Piazza. 

In [ ]:
# Import necessary modules for this assignment 
# Do not modify this cell 
from collections import defaultdict, Counter
import operator
import random
from typing import List, Dict, Union
import numpy as np
import matplotlib.pyplot as plt

from util import * #helper functions for this assignment located in util.py

**Note:** In this assignment, you are **NOT** allowed to import or use any other packages outside the Python standard and the ones we imported above.

This means you should not use any other functions from `spaCy`, `NLTK`, `gensim`, or `scikit-learn`, even though those are provided in the conda environment we set up for you. If your solution uses any such extra dependencies it will fail the autograder.

In [ ]:
# Set the random seed for numpy 
np.random.seed(42)

## 1B. Data exploration 

In this assignment, we will be using the `triage` dataset, which we also used for HW2. Feel free to return to HW2 for a longer description of this dataset. 

In [ ]:
# Load this data via the function from util.py
triage_dataset = load_data("./data/triage")

In [ ]:
# Re-run this cell several times to look at different examples 
random_index = random.randint(0, len(triage_dataset.train))
print(f"Training example {random_index}:")
print(f"Label(y) = {triage_dataset.train[random_index].label}")
tokens = triage_dataset.train[random_index].words
text = " ".join(tokens)
print(f"Text = {text}")
print()
print(f"Tokens = {tokens}")

Unlike HW2, for logistic regression we will represent the input documents as a matrix (specifically a 2-D np.ndarray) for which rows are documents and columns are counts of words. Inspect the `transform_examples_to_arrays()` function in `util.py` to see how we use `CountVectorizer` from `sklearn` to do this.

In [ ]:
data = transform_examples_to_arrays(triage_dataset.train, triage_dataset.dev)

In [ ]:
data.keys()

In [ ]:
# Number of words in our training vocabulary
data['x_columns_as_words'].shape

In [ ]:
# ten words from out training vocabulary 
data['x_columns_as_words'][100:110]

In [ ]:
# Training X is ~21K documents (rows)
# With counts for each word in our vocabulary (columns)
data['X_train'].shape

In [ ]:
# Training Y labels 
data['Y_train'].shape

In [ ]:
# First 10 labels in our training dataset 
data['Y_train'][0:10]

## 2. Your solution 

### 2A. Logistic function 

One of the building blocks of `LogisticRegression` is the `Logistic` function which we described during lecture. Implement the logistic function below. 

In [ ]:
def logistic_function(z: np.ndarray) -> np.ndarray:
    """
    Impelmentation of the logistic function 

    Args:
        z: A numpy array.
    Returns:
        The numpy array z with the logistic function applied element-wise

    Hints
     - You should be using numpy and array broadcasting here 
     - np.exp may be helpful 
    """
    # TODO: implement your solution here 
    # CODE START
    raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
    # CODE END

Once you are done with your implementation, try visualizing it to
double-check the implementation. Does the graph below look reasonable? 

In [ ]:
# Create a numpy array containing evenly separated numbers, from -10 to 10 
# (inclusive)
z = np.arange(-10, 10, .01)

# Call our sigmoid function on the newly created array
output = logistic_function(z)

# Plot
plt.plot(z, output, color='blue', lw=2)
plt.xlabel('z')
plt.ylabel('logistic_function(z)');

### 2B. Negative log likelihood (NLL) loss function 

Another building block of binary `LogisticRegression` is the `Negative Log Likelihood` loss function. Implement this loss function (as discussed in lecture) below. 

In [ ]:
def NLLLoss(pred_prob: np.ndarray, y_true: np.ndarray, epsilon=1e-20) -> float:
    """
    
    Args: 
        - pred_prob: A 1-D vector of your model's predicted probability of y=1
        - y_true: A 1-D vector of the true class labels 
        - epsilon (optional): a very small number added to the input every time np.log is called
            for numerical stability 
        
    Returns: 
        - float: The negative log likelihood
        
    Hints:
    - If you run into "RuntimeWarning: divide by zero encountered in log" 
        issues, try adding a very small number epsilon (like 1e-10) to the 
        input any time you call np.log(). This will ensure that the input to 
        the log is never exactly 0, which gives an undefined result.
    - Recall from lecture, your solution should average over the number of examples 
    """
    assert pred_prob.shape == y_true.shape #inputs must be the same size 
    
    # TODO: implement your solution here 
    # CODE START
    raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
    # CODE END 

The loss is supposed to represent how far our prediction is from the true label. In other words, if our prediction is very far from the truth, the loss should be very high. If our prediction gets closer to the true value, the loss should drop towards 0. Let's consider a few cases to understand how the loss should change on specific input.

In [ ]:
def print_loss_single_example(pred_prob_single, y_true_single):
    print("Predicted Prob = {}, True = {} : Loss = {}".format(
          pred_prob_single, y_true_single, NLLLoss(np.array([pred_prob_single]),
                                        np.array([y_true_single]))))
print_loss_single_example(0.0, 0)
print_loss_single_example(0.0, 1)
print_loss_single_example(0.1, 1)
print_loss_single_example(0.3, 1)
print_loss_single_example(0.5, 1)
print_loss_single_example(0.7, 1)
print_loss_single_example(0.9, 1)
print_loss_single_example(0.99, 1)
print_loss_single_example(0.999999, 1)
print_loss_single_example(1, 1)

In [ ]:
# Make sure your impelmentation works for vectors as well 
pred_prob = np.array([0.8, 0.2, 0.7])
y_true = np.array([1, 0, 1])
NLLLoss(pred_prob, y_true)

### 2C. Gradient descent 

During lecture, we derived the gradient of the negative log likelihood. Implement that gradient in the function below. 

In [ ]:
def gradient_nll(X: np.ndarray, 
                 y: np.ndarray, 
                 theta: np.ndarray) -> np.ndarray: 
    """
    Returns the gradient of the negative log likelihood (NLL) 
    with respect to the inputs: 
        - X: shape=(num_examples, num_words)
        - y: shape=(num_examples,)
        - theta: shape=(num_words,)
        
    Hints: 
        - At some point during this function, you should call logistic_function 
            which you implemented above
        - You should not have any for-loops in this code. You should use 
            numpy array operations and broadcasting. Otherwise your implementation
            will run significantly slower. 
        - Section 5.6.4 from the J&M textbook will be helpful here 
    """
    # TODO: implement your solution here 
    # CODE START
    raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
    # CODE END 

We give you a very small toy dataset in the cell below. We encourage you to calculate the gradient by hand and then check if your implementation matches what you get on paper. 

In [ ]:
X = np.array([[1, 2, 1], 
              [0, 1, 1]])
y = np.array([1, 0])
theta = np.array([-1, 0, 1])
gradient = gradient_nll(X, y, theta)
print('Gradient =', gradient)
assert gradient.shape[0] == theta.shape[0]

Now that you have written the function for the gradient, we have implemented mini-batch stochastic gradient descent for you below. 

In [ ]:
# NO NEED TO MODIFY THIS CELL 
def stochastic_gradient_descent(X: np.ndarray,
                     Y: np.ndarray,
                     batch_size: int = 2000,
                     alpha: float = 0.5,
                     num_iterations: int = 1000,
                     print_every: int = 100,
                     early_stopping_value: float = 1e-8, 
                     l2_regularization_strength: float = 0.01) -> np.ndarray:
    """
    Runs mini-batch stochasitc gradient descent on the provided data and returns the resulting
    trained weight vector. 

    Args:
        - X: A numpy array of shape (num_examples, num_words) containing
           the training data.
        - Y: A numpy array of shape (num_examples,) containing the training
            labels.
        - batch_size: The number of examples in each batch.
        - alpha: The learning rate for gradient descent.
        - num_iterations: The number of iterations to run gradient descent for.
        - print_every: How often (after how many iterations) to print the
                    loss and iteration number.
        - early_stopping_value: The early stopping condition. When the absolute change
                 in the loss is less than early_stopping_value, gradient descent will
                 stop early.
        - l2_regularization_strength: the constant in front of the L2 penalty term. Set to 0 
            if you would like to turn off regularization. 

    Returns:
        np.ndarray: The learned weight vector theta 
        
    Notes: 
        - Remember we are "absorbing the bias" like we discussed during lecture 
    """
    assert X.shape[0] == Y.shape[0]
    
    # add ones to X to "absorb the bias"
    stack = np.ones((X.shape[0], 1))
    X = np.concatenate((X, stack), axis=1)
    
    # initalize the weight vector 
    theta = np.zeros((X.shape[1],))
    
    loss = 0
    for i in range(num_iterations):
        #create batches 
        if batch_size >= X.shape[0]: 
            X_batch = X
            Y_batch = Y
        else: #randomly choose batch_size number of examples 
            batch_indices = np.random.randint(X.shape[0], size=batch_size)
            X_batch = X[batch_indices]
            Y_batch = Y[batch_indices]
        
        # One step through gradient descent 
        Y_batch_pred_prob = logistic_function(np.dot(X_batch, theta))
        d_theta = gradient_nll(X_batch, Y_batch, theta)
        # Make sure to account for L2 regularization in the gradient update equation
        theta -= (alpha * d_theta +2*l2_regularization_strength*theta)
        prev_loss = loss 
        loss = NLLLoss(Y_batch_pred_prob, Y_batch)
        # Add L2 regularization to the loss 
        loss += l2_regularization_strength * np.sum(np.square(theta))
        
        # early stopping condition 
        if abs(prev_loss - loss) < early_stopping_value:
            break
            
        #print some diagnostics 
        if (i+1) % print_every == 0:
            Y_batch_pred = np.zeros(Y_batch.shape)
            Y_batch_pred[Y_batch_pred_prob >= 0.5] = 1
            accuracy = np.mean(Y_batch_pred == Y_batch)
            print(f"Iteration {i + 1}/{num_iterations}: Batch Accuracy: {np.round(accuracy,2)},  Batch Loss = {np.round(loss,3)}")
    return theta 

### 2D. LogisticRegressionClassifier

Complete the implementation of the `LogisticRegessionClassifier` below.

You are welcome to create additional helper functions *within* the class if your implementation requires it. However, any functions you write outside of the `LogisticRegessionClassifier` class cannot be accessed by the autograder and may cause it to fail. 

In [ ]:
# TODO: complete the implementation below 
class LogisticRegressionClassifier:
    """
    Implements Naive Bayes Classifier 
    Includes Laplace smoothing (add-1 smoothing) during training 
    """
    def __init__(self,
                 batch_size: int = 2000,
                 alpha: float = 0.5,
                 num_iterations: int = 1000,
                 print_every: int = 100,
                 early_stopping_value: float = 1e-8,
                 l2_regularization_strength: float = 0.01):
        
        # Parameters used by stochastic gradient descent 
        self.batch_size = batch_size
        self.alpha = alpha
        self.num_iterations = num_iterations
        self.print_every = print_every
        self.early_stopping_value = early_stopping_value
        self.l2_regularization_strength = l2_regularization_strength
        
        # TODO: add other data structures needed for predict() or train()
        # CODE START
        pass 
        # CODE END

    def train(self, X_train: np.ndarray, y_train:np.ndarray) -> None:
        """
        This method inputs the training data (X, y) both of which are 
        numpy arrays and trains the Logistic Regression classifier.  
        
        Hints: 
            - You should call stochastic_gradient_descent() at some
                point in this function 
            - You should pass in the parameters we initialize above in __init__ that are used by 
                stochastic gradient descent 
        """
        # TODO: implement your solution here 
        # CODE START
        raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
        # CODE END
             
    def predict(self, X_test: np.ndarray) -> Union[List[int], List[float]]:
        """
        This method inputs the examples X_test (a numpy array)
        
        This method returns a list of int variables that are the predicted
        labels (e.g. 0 or 1)
        
        You should use a decision threshold of P(y=1|x) > 0.5 to predict 1 as the label for y.
            
        Hints: 
            - At some point in this method, you should use the theta vector 
            you computed earlier in train()
            - Remember that during stochastic_gradient_descent() we "absorb the bias" for theta.
                How do you modify X_test as a result? 
        """
        # TODO: implement your solution here 
        # CODE START
        raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
        # CODE END

    def get_weights(self) -> np.ndarray:
        """
        TODO: Implement a method to return the trained weights.

        Returns:
            np.ndarray: Trained weights
            
        Hint: 
            - Don't overthink this. This method should be one line of code if you set-up 
            your class correclty 
        """
        # CODE START
        raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
        # CODE END

#### Debugging on "toy" corpus 

Like most real-world NLP systems, it can be helpful to examine the correctness of our code on a small "toy" dataset that we can analytically calculate the answers for.

Below, we implemented the example from lecture with toy corpus words "repair, "beyond" and "good". Use this to check your implementation. 

In [ ]:
# No need to modify this cell
toy_X_train = np.array([[1, 0, 0], 
                        [0, 1, 1], 
                        [1, 1, 0], 
                        [0, 0, 1]])
toy_y_train = np.array([0, 1, 0, 1])

In [ ]:
# Call to check your implementation 
lr_classifier_toy = LogisticRegressionClassifier(batch_size=4, num_iterations=100, 
                                                 print_every=10, l2_regularization_strength=0)
lr_classifier_toy.train(toy_X_train, toy_y_train)

With 100 epochs, you should be seeing `Batch Loss` values of less than `0.05` in the cell above. 

In [ ]:
y_pred = lr_classifier_toy.predict(toy_X_train)
print('Models predictions for toy_X_train = ', y_pred)

If you trained your model correctly, you should be getting an accuracy of 100\% for this toy corpus training data. Knowing this, how can you check that `y_pred` in the cell above is giving you the correct answer? 

## 3. Evaluation 

### 3A. Accuracy 

We will evaluate your classifier on the `triage` dataset, the same one that we evaluated on Naive Bayes in HW3. This will give you an opportunity to compare the performance of the two classifiers. 

In [ ]:
# Make sure you have run the cells in Part 1B that load the triage data 
data.keys()

In [ ]:
# DO NOT MODIFY THIS CELL 
lr_classifier = LogisticRegressionClassifier(batch_size=2000, num_iterations=1000, print_every=100, alpha=1, l2_regularization_strength=0.0005)
lr_classifier.train(data['X_train'], data['Y_train'])

In our reference implementation, the cell above takes about 2 seconds to run. If yours is taking longer than several minutes, you may want to return to the functions you wrote earlier and make sure they are vectorized and using `numpy` effectively. 

In [ ]:
# DO NOT MODIFY THIS CELL 
Y_pred = lr_classifier.predict(data['X_dev'])
accuracy = np.mean(Y_pred == data['Y_dev']) 
print('Development accuracy = ', accuracy)

### 3B. Inspecting word weights 

For Logistic Regression, we are able to inspect the individual weights, $\theta_k$, learned for each word in the vocabulary. Let's inspect these to see what our model learned for the `triage` data.  

In [ ]:
# NO NEED TO MODIFY THIS CELL 

words = data['x_columns_as_words']
weights = lr_classifier.get_weights()

words_to_weights = [(words[i], weights[i])
                    for i in range(len(words))]

In [ ]:
# NO NEED TO MODIFY THIS CELL 

print('Highest weights on words')
print('==='*15)
top_10_words = sorted(words_to_weights,
                   key=operator.itemgetter(1), reverse=True)[:10]
for word, weight in top_10_words:
    print("{0:<15} weight = {1:.2f}".format(word, weight))

In [ ]:
# NO NEED TO MODIFY THIS CELL 

print('Lowest weights on words')
print('==='*15)

bottom_10_words = sorted(words_to_weights,
                            key=operator.itemgetter(1))[:10]
for word, weight in bottom_10_words:
    print("{0:<15} weight = {1:.2f}".format(word, weight))

### 3C. Tradeoffs


You have now implemented both Logistic Regression and Naive Bayes and evaluated the trained classifiers on the same datsaet. What do you see as the tradeoffs of these two models' assumptions, ease of implementation, interpretability, and predictive performance?

(We are expecting at minimum *three* complete sentences for full credit. There may be more than one correct answer for this question.) 

*Hints*: 
- Compare the interprebability and validity of the "highest weights on words" above for Logistic Regression to the p(label|word) you learned for Naive Bayes.
- What aspects of the Naive Bayes implementation last week versus the Logistic Regression implementation this week were easier? More reliable? 
- Discuss the predictive performance (accuracies) of the two models. 

**Part 3C Answer:**

*DELETE AND PUT YOUR ANSWER HERE.* 

## (Optional) 4. Extra Credit 

*Extra credit can only help you and will not hurt you. At the end of the semester, if you have a borderline grade, your grade could be increased given your efforts on extra credit. This section is intended to be open-ended and challenge you. We suggest you only attempt this section after you have completed all other parts and are satisifed with your submission.*

Above, we have implemented the most basic version of Logistic Regression. Try to extend it and/or make it better. Some suggestions to try:
- Extend for multi-class (multinomial) logistic regression (J&M Ch. 5.3)
- Extend your classifier to also output the predicted probabilities. Then evaluate your classifier on calibration. See the calibration section of [Katie's tutorial for social scientists](https://colab.research.google.com/drive/1ulQSwlSlWTEglzBGVQXKstueIaX5Gm1f?usp=sharing). 
- In stochastic gradient descent, instead of using a single learning rate, use an adaptative learning rate, e.g. [Adam](https://arxiv.org/abs/1412.6980). 
- Implement L1 regularization (J&M Ch. 5.7)
- Change the decision threshold from 0.5 to something else and evaluate how this changes your precision and recall.
- Use a grid search and cross validation to tune all hyperparameters. 
- Anything else you want to try! 

For an extra credit submission please create a separate `.ipynb` file and submit to `Extra Credit: HW 3` submission on Gradescope. 

This is to ensure your extra credit work does not interfere with or inhibit your main submission. 

## Submission 

**Processing reporting.** Please record your answers to the questions below by writing directly in this Markdown cell.

If you talked with any of your classmates on this assignment please list their names here:

*DELETE AND PUT YOUR ANSWER HERE.*

Approximately how much time did you spend on this assignment:

*DELETE AND PUT YOUR ANSWER HERE.*

**Download zip.** Once you're satsified with your solution, save this file and run the cell below to automatically zip your file. This will produce `submission.zip` in the same folder as this file (same folder as `hw3.ipynb`). 

Submit `submission.zip` to Gradescope. 

*Note:* This script assumes that you have the `zip` utility installed and you can use `bash` on your system. If the cell below does not work you may need to zip your file manually. 

In [ ]:
%%bash

if [[ ! -f "./hw3.ipynb" ]]
then
    echo "WARNING: Did not find notebook in Jupyter working directory. Manual solution: go to File->Download .ipynb to download your notebok and other files, then zip them locally."
else
    echo "Found notebook file, creating submission zip..."
    zip -r submission.zip hw3.ipynb
fi